In [1]:
import pandas as pd
import numpy as np

from bokeh.io import show, output_notebook, push_notebook, output_file

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import all_palettes

from bokeh.models.widgets import CheckboxGroup, RangeSlider, DataTable, DateFormatter, TableColumn
from bokeh.layouts import column, row, WidgetBox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [2]:
def make_plot(source):


    p = figure(title="Sacremento Dataset", x_axis_label='Longitude', y_axis_label='Latitude')
    p.circle(x="longitude",y="latitude",size=10,color='color',legend='type',alpha=0.5,source=source)


    p.add_tools(HoverTool(
            tooltips=[
                    ( 'street', '@street'),
                    ( 'price', '$@price{,f}'),
                    ( 'sq__ft', '@sq__ft{,f}ft^2'),
                    ( 'beds', '@beds'),
                    ( 'baths', '@baths'),
                    ]

        ))
    
    hover = p.select(dict(type=HoverTool))

    return p


def make_dataset(df, selected_type, price_range, baths_range, sq_ft_range, beds_range):


    new_df = pd.DataFrame()
    for i, house in enumerate(selected_type):
        mask = df['type']==house
        new_df = new_df.append(df[mask], ignore_index=True)

        
    mask = (new_df['price']>=price_range[0]) & (new_df['price']<=price_range[1])
    new_df = new_df[mask]
    
    mask = (new_df['baths']>=baths_range[0]) & (new_df['baths']<=baths_range[1])
    new_df = new_df[mask]
    
    mask = (new_df['sq__ft']>=sq_ft_range[0]) & (new_df['sq__ft']<=sq_ft_range[1])
    new_df = new_df[mask]
    
    mask = (new_df['beds']>=beds_range[0]) & (new_df['beds']<=beds_range[1])
    new_df = new_df[mask]

    source = ColumnDataSource(new_df)

    return source


# Update function takes three default parameters
def update(attr, old, new):
    # Get the list of carriers for the graph
    selected_type = [housing_selection.labels[i] for i in housing_selection.active]
    price_range = [range_slider_price.value[0], range_slider_price.value[1]]
    baths_range = [range_slider_baths.value[0], range_slider_baths.value[1]]
    sq_ft_range = [range_slider_sq_ft.value[0], range_slider_sq_ft.value[1]]
    beds_range = [range_slider_beds.value[0], range_slider_beds.value[1]]

    # Make a new dataset based on the selected carriers and the 
    # make_dataset function defined earlier
    new_src = make_dataset(df, selected_type, price_range, baths_range, sq_ft_range, beds_range)
    # Update the source used in the quad glpyhs
    source.data.update(new_src.data)

In [3]:
def modify_doc(doc):
  
    housing_selection.on_change('active', update)    
    range_slider_price.on_change('value', update)
    range_slider_baths.on_change('value', update)
    range_slider_beds.on_change('value', update)
    range_slider_sq_ft.on_change('value', update)
    
    doc.add_root(row(p,column(data_table,controls)))

In [4]:
df = pd.read_csv("Sacramentorealestatetransactions.csv")
names = df["type"].unique();

color_temp = all_palettes['Viridis'][names.size];

colormap = {}    
for i, j in zip(names, color_temp):
    colormap[i] = j

colors_obj = [colormap[x] for x in df['type']]
df['color'] = colors_obj

# Type of House Widget
housing_selection = CheckboxGroup(labels=list(names),
                                      active = list(range(0,names.size)))
selected_type = [housing_selection.labels[i] for i in housing_selection.active]


# RangeSlider for price
range_slider_price = RangeSlider(start=df['price'].min(), end=df['price'].max(), value=(df['price'].min(),df['price'].max()), step=1000, title="Price")
price_range = [range_slider_price.value[0], range_slider_price.value[1]]


# RangeSlider for baths
range_slider_baths = RangeSlider(start=df['baths'].min(), end=df['baths'].max(), value=(df['baths'].min(),df['baths'].max()), step=1, title="Number of Baths")
baths_range = [range_slider_baths.value[0], range_slider_baths.value[1]]

# RangeSlider for sq_ft
range_slider_sq_ft = RangeSlider(start=df['sq__ft'].min(), end=df['sq__ft'].max(), value=(df['sq__ft'].min(),df['sq__ft'].max()), step=100, title="Square Footage")
sq_ft_range = [range_slider_sq_ft.value[0], range_slider_sq_ft.value[1]]

# RangeSlider for beds
range_slider_beds = RangeSlider(start=df['beds'].min(), end=df['beds'].max(), value=(df['beds'].min(),df['beds'].max()), step=1, title="Number of Beds")
beds_range = [range_slider_beds.value[0], range_slider_beds.value[1]]

# Hovertool

controls = WidgetBox(housing_selection, range_slider_price,range_slider_sq_ft, range_slider_baths, range_slider_beds)
    
source = make_dataset(df, selected_type, price_range, baths_range, sq_ft_range, beds_range)
p = make_plot(source)

In [5]:
columns = [
        TableColumn(field="street", title="Address"),
        TableColumn(field="price", title="Prices"),
    ]
data_table = DataTable(source=source, columns=columns, width=400, height=280)

In [6]:
show(modify_doc)